In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

TensorFlow 2.x selected.


In [0]:
import pandas as pd 

In [0]:
url = "https://github.com/motazsaad/work-online-ds/raw/master/data/IRIS.csv"

dataframe = pd.read_csv(url)
dataframe.head(2)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


# Steps: 
# 2. split data
# 3. load as tensor slice data
# 4. shuffle and batch 
# 5. apply sequential model

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
dataframe = dataframe.apply(le.fit_transform)
dataframe.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
145,24,9,28,19,2
146,20,4,26,15,2
147,22,9,28,16,2
148,19,13,30,19,2
149,16,9,27,14,2


In [0]:
train, test = train_test_split(dataframe, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


121 train examples
14 validation examples
15 test examples


In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target, shuffle=True, batch_size=100):
  dataframe = dataframe.copy()
  labels = dataframe.pop(target)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 10 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(dataframe=train, target='species', shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(dataframe=val,  target='species', shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(dataframe=test,  target='species', shuffle=False, batch_size=batch_size)

In [0]:
dataframe.keys()

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [0]:
feature_columns = []

# numeric cols
for col in dataframe.keys():
  if col == 'species':
    continue
  feature_columns.append(feature_column.numeric_column(col))

In [0]:
feature_columns

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.DenseFeatures(feature_columns))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [0]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100
13/13 [==============================] - 0s 7ms/step - loss: 8.9675e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 9.2841e-04 - accuracy: 1.0000 - val_loss: 1.1570 - val_accuracy: 0.8571
Epoch 3/100
13/13 [==============================] - 0s 3ms/step - loss: 9.6226e-04 - accuracy: 1.0000 - val_loss: 1.2009 - val_accuracy: 0.8571
Epoch 4/100
13/13 [==============================] - 0s 3ms/step - loss: 9.1855e-04 - accuracy: 1.0000 - val_loss: 1.1720 - val_accuracy: 0.8571
Epoch 5/100
13/13 [==============================] - 0s 4ms/step - loss: 8.6740e-04 - accuracy: 1.0000 - val_loss: 1.1882 - val_accuracy: 0.8571
Epoch 6/100
13/13 [==============================] - 0s 4ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.1038 - val_accuracy: 0.8571
Epoch 7/100
13/13 [==============================] - 0s 4ms/step - loss: 8.4866e-04 - accuracy: 1.0000 - val_loss: 1.2185 - va

In [0]:
model.evaluate(test_ds)

2/2 [==============================] - 0s 4ms/step - loss: 0.2379 - accuracy: 0.9333


[0.23790750186890364, 0.93333334]